In [1]:
import numpy as np
import scipy as scipy
from scipy.optimize import minimize
import timeit

# Try Solving 6 Ancestry problem, multiple SNPs

In this notebook we attempt to solve the following constrained, quadratic optimization problem:

$$\min_{\pi \in \mathbb{R}^6} f(\pi)=\sum_{i=1}^{N}(a_{1,i}\pi_1+a_{2,i}\pi_2+a_{3,i}\pi_3+a_{4,i}\pi_4+a_{5,i}\pi_5+a_{6,i}\pi_6-\tilde{a}_i)^2$$

$$\text{subject to:} \quad \pi_1+\pi_2+\pi_3+\pi_4+\pi_5+\pi_6=1 \quad \pi_1\geq 0 \quad \pi_2 \geq 0 \quad \pi_3 \geq 0 \quad \pi_4 \geq 0 \quad \pi_5 \geq 0 \quad \pi_6 \geq 0,$$

where $a_{j,i} \in \mathbb{R}$, $j=1,2,3,4,5,6$; $i=1,\ldots N$ and $\tilde{a}_i \in \mathbb{R}$, $i =1, \ldots, N$ are quantities obtained from a genetics simulation. The $a_{1,i}$'s correspond to the observed allele frequency in ancestry 1; the $a_{2,i}$'s correspond to the observed allele frequency in ancestry 2; the $a_{3,i}$'s correspond to the observed allele frequency in ancestry 3; the $a_{4,i}$'s correspond to the observed allele frequency in ancestry 4; the $a_{5,i}$'s correspond to the observed allele frequency in ancestry 5; the $a_{6,i}$'s correspond to the observed allele frequency in ancestry 6; the $\tilde{a}_i$'s corespond to the observed  total allele frequency in the population. Here, $N$ is the number of SNPs.

In [2]:
# Simulate 10^6 SNPS


M=1000000

a_1 = np.random.uniform(low=0, high=0.6, size=(M,1))
a_2 = np.random.uniform(low=0.2, high=0.5, size=(M,1))
a_3 = np.random.uniform(low=0.1, high=0.5, size=(M,1))
a_4 = np.random.uniform(low=0, high=0.5, size=(M,1))
a_5 = np.random.uniform(low=0.1, high=0.5, size=(M,1))
a_6 = np.random.uniform(low=0.1, high=0.6, size=(M,1))
a_t = .155*a_1 + .245*a_2 +.3*a_3+.2*a_4 +.01*a_5+.09*a_6
# By choosing the mixing proportions, this makes the "answer" pi_1=.155, pi_2=.245, pi_3=.3, pi_4=.2, pi_5=.01, pi_6=.09

In [3]:
# This is the objective function!

def function(x):
    return np.sum((a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t)**2,axis=0)

In [4]:
# This is a feasible initial point since its components add to 1 and are positive.

x_t=np.array((1/6,1/6,1/6,1/6,1/6,1/6))

# Make sure function works by computing f(x_t)

print(function(x_t))

[763.52671617]


In [5]:
# Here is the gradient of the objective function



def gradfun(x):
    return np.array((np.sum(2*a_1*(a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t),axis=0),
                     np.sum(2*a_2*(a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t),axis=0),
                     np.sum(2*a_3*(a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t),axis=0),
                     np.sum(2*a_4*(a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t),axis=0),
                     np.sum(2*a_5*(a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t),axis=0),
                     np.sum(2*a_6*(a_1*x[0]+a_2*x[1]+a_3*x[2]+a_4*x[3]+a_5*x[4]+a_6*x[5]-a_t),axis=0)))

In [6]:
#print(gradfun(x_t))
print(gradfun((.155,.245,.3,.2,.01,.09)))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


## SLSQP

In [7]:
cons = ({'type': 'eq', 'fun': lambda x:  x[0] + x[1] + x[2] + x[3] + x[4] +x[5] -1},
        {'type': 'ineq', 'fun': lambda x: x[0]},
        {'type': 'ineq', 'fun': lambda x: x[1]},
        {'type': 'ineq', 'fun': lambda x: x[2]},
        {'type': 'ineq', 'fun': lambda x: x[3]},
        {'type': 'ineq', 'fun': lambda x: x[4]},
        {'type': 'ineq', 'fun': lambda x: x[5]})

bnds = ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None))

In [8]:
start = timeit.default_timer()

print(scipy.optimize.minimize(function, x_t, method='SLSQP', jac=gradfun, bounds=bnds, constraints=cons))

print('Correct ans', (.155,.245,.3,.2,.01,.09))

stop = timeit.default_timer()

print('Time: ', stop - start)

     fun: 2.0653274249889937e-08
     jac: array([ 0.01221792,  0.00786982, -0.01559756,  0.00328881, -0.00188523,
       -0.03368706])
 message: 'Optimization terminated successfully.'
    nfev: 29
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([0.15500025, 0.24500076, 0.29999953, 0.20000014, 0.01000004,
       0.08999927])
Correct ans (0.155, 0.245, 0.3, 0.2, 0.01, 0.09)
Time:  13.05436010759924
